# Exploratory Data Analysis

We have data on happiness, democracy, and __. Let's first take a look at all the data and make something interesting out of some horrendous datasets.

In [4]:
import pandas as pd
import numpy as np
# import geopandas as gpd
# import altair as alt
# from altair_data_server import data_server
import os 
import re

# Happiness Data

## Useful Links
* [geoJSON file downloads](https://gadm.org/download_country.html)

In [5]:
path = "Datasets/Happiness_Data/World_Happiness_Reports"
# The pattern looks for a sequence of exactly four digits
pattern = r"\d{4}"

country_sets = []
dataframes = []

# loop through all the happiness.csv files for each year and gather the happiness scores
for filename in os.scandir(path):
    print("Working on: " + filename.path)

    # the current csv file we're working on 
    curr_df = pd.read_csv(filename)
    
    # some countries had an asterisk next to their name, so remove those
    curr_df['Country'] = curr_df['Country'].str.replace('*', '', regex=True)

    # there are differnt number of countries in different files, so we want to collect all the countries 
    country_set = set(curr_df['Country'].tolist())
    country_sets.append(country_set)

    # add "_year" after the Happiness Score for columns
    match = re.search(pattern, filename.path)
    year = match.group(0) if match else None 
    curr_df.rename(columns={"Happiness Score":"Happiness Score " + year,
                            "Social support":"Social Support " + year,
                            "Freedom to make life choices": "Freedom " + year,
                            "Generosity": "Generosity " + year,
                            'Perceptions of corruption':"Corruption " + year
                            }, inplace=True)
    dataframes.append(curr_df)
    
    print(f"\tColumns: {curr_df.columns}")
# use sets to just get the unique countries
common_countries = set.union(*country_sets)


Working on: Datasets/Happiness_Data/World_Happiness_Reports/2020.csv
	Columns: Index(['Country', 'Regional indicator', 'Happiness Score 2020',
       'Standard error of ladder score', 'upperwhisker', 'lowerwhisker',
       'Logged GDP per capita', 'Social Support 2020',
       'Healthy life expectancy', 'Freedom 2020', 'Generosity 2020',
       'Corruption 2020', 'Ladder score in Dystopia',
       'Explained by: Log GDP per capita', 'Explained by: Social support',
       'Explained by: Healthy life expectancy',
       'Explained by: Freedom to make life choices',
       'Explained by: Generosity', 'Explained by: Perceptions of corruption',
       'Dystopia + residual'],
      dtype='object')
Working on: Datasets/Happiness_Data/World_Happiness_Reports/2021.csv
	Columns: Index(['Country', 'Regional indicator', 'Happiness Score 2021',
       'Standard error of ladder score', 'upperwhisker', 'lowerwhisker',
       'Logged GDP per capita', 'Social Support 2021',
       'Healthy life expecta

In [7]:
# Filter each dataset to only include rows with countries in the common countries set
filtered_dataframes = []
for df in dataframes:
    filtered_df = df[df["Country"].isin(common_countries)]
    filtered_dataframes.append(filtered_df)

In [8]:
# create the happiness dataframe
pattern = r"Happiness Score"
happiness_df = filtered_dataframes[0][['Country']]
for i, df in enumerate(filtered_dataframes):
    # Extract the country and happiness score columns
    filtered_columns = df.filter(regex=pattern)

    country_happiness_df = pd.concat([df['Country'], filtered_columns], axis=1)

    # Merge the result DataFrame with the country_happiness_df on the 'Country' column
    happiness_df = happiness_df.merge(country_happiness_df, on='Country', how='right')

# Split the DataFrame into the first column and the remaining columns
first_column = happiness_df.iloc[:, 0:1]
remaining_columns = happiness_df.iloc[:, 1:]

# Sort the remaining columns
sorted_remaining_columns = remaining_columns.sort_index(axis=1)

# Concatenate the first column with the sorted remaining columns
happiness_df = pd.concat([first_column, sorted_remaining_columns], axis=1)

happiness_df['Happiness Score 2022'] = happiness_df['Happiness Score 2022'].str.replace(',','.')
happiness_df['Happiness Score 2022'] = pd.to_numeric(happiness_df['Happiness Score 2022'], errors ='ignore').astype(pd.Float64Dtype())

happiness_df

,Country,Happiness Score 2015,Happiness Score 2016,Happiness Score 2017,Happiness Score 2018,Happiness Score 2019,Happiness Score 2020,Happiness Score 2021,Happiness Score 2022
0,Norway,7.522,7.498,7.537,7.594,7.554,7.4880,7.392,7.365
1,Denmark,7.527,7.526,7.522,7.555,7.600,7.6456,7.620,7.636
2,Iceland,7.561,7.501,7.504,7.495,7.494,7.5045,7.554,7.557
3,Switzerland,7.587,7.509,7.494,7.487,7.480,7.5599,7.571,7.512
4,Finland,7.406,7.413,7.469,7.632,7.769,7.8087,7.842,7.821
...,...,...,...,...,...,...,...,...,...
150,Rwanda,3.465,3.515,3.471,3.408,3.334,3.3123,3.415,3.268
151,Syria,3.006,3.069,3.462,3.462,3.462,NaN,NaN,<NA>
152,Tanzania,3.781,3.666,3.349,3.303,3.231,3.4762,3.623,3.702
153,Burundi,2.905,2.905,2.905,2.905,3.775,NaN,NaN,<NA>


In [9]:
# Let's make a choropleth map, so we need to upload geojson files
countries_filepath = 'Datasets/geoJSONs/countries.geojson'
countries_gdf = gpd.read_file(countries_filepath)
countries_gdf = countries_gdf.rename(columns={'ADMIN':'Country'})

NameError: name 'gpd' is not defined

In [6]:
# Merge happiness data so we can look at the different levels of happiness in different countries across different years
happiness_countries_merged_df = countries_gdf.merge(happiness_df, on='Country', how = 'right')
happiness_countries_merged_df

,Country,ISO_A3,ISO_A2,geometry,Happiness Score_2015,Happiness Score_2016,Happiness Score_2017,Happiness Score_2018,Happiness Score_2019,Happiness Score_2020,Happiness Score_2021,Happiness Score_2022
0,Norway,NOR,NO,"MULTIPOLYGON (((3.38258 -54.44931, 3.36451 -54...",7.522,7.498,7.537,7.594,7.554,7.4880,7.392,7.365
1,Denmark,DNK,DK,"MULTIPOLYGON (((11.25603 54.95458, 11.30348 54...",7.527,7.526,7.522,7.555,7.600,7.6456,7.620,7.636
2,Iceland,ISL,IS,"MULTIPOLYGON (((-20.26964 63.40913, -20.27953 ...",7.561,7.501,7.504,7.495,7.494,7.5045,7.554,7.557
3,Switzerland,CHE,CH,"MULTIPOLYGON (((8.61744 47.75732, 8.62984 47.7...",7.587,7.509,7.494,7.487,7.480,7.5599,7.571,7.512
4,Finland,FIN,FI,"MULTIPOLYGON (((22.44370 59.85147, 22.42774 59...",7.406,7.413,7.469,7.632,7.769,7.8087,7.842,7.821
...,...,...,...,...,...,...,...,...,...,...,...,...
150,Rwanda,RWA,RW,"MULTIPOLYGON (((30.47179 -1.06684, 30.46386 -1...",3.465,3.515,3.471,3.408,3.334,3.3123,3.415,3.268
151,Syria,SYR,SY,"MULTIPOLYGON (((42.23683 37.28630, 42.26722 37...",3.006,3.069,3.462,3.462,3.462,NaN,NaN,<NA>
152,Tanzania,NaN,NaN,None,3.781,3.666,3.349,3.303,3.231,3.4762,3.623,3.702
153,Burundi,BDI,BI,"MULTIPOLYGON (((30.41507 -2.31309, 30.41848 -2...",2.905,2.905,2.905,2.905,3.775,NaN,NaN,<NA>


In [7]:
# Congo was originally broken into "Congo (Brazzaville)" and "Congo (Kinshasa)", I couldn't find a geojson file for them, but I could for Congo, so I just took the average. 
congo_rows = happiness_countries_merged_df[happiness_countries_merged_df['Country'].isin(['Congo (Brazzaville)', 'Congo (Kinshasa)'])]
avg_row = congo_rows.mean(numeric_only=True)

# Create a new row with the average values
new_row = congo_rows.iloc[0].copy()
new_row[new_row.index.str.startswith('Happiness Score ')] = avg_row

# Set the new country name
new_row['Country'] = 'Congo'

# Remove the original rows from the DataFrame
happiness_countries_merged_df = happiness_countries_merged_df[~happiness_countries_merged_df['Country'].isin(['Congo (Brazzaville)', 'Congo (Kinshasa)'])]

# Add the new row with the average values to the DataFrame
happiness_countries_merged_df = happiness_countries_merged_df.append(new_row, ignore_index=True)
happiness_countries_merged_df

/var/folders/fg/mmz1wr455573gbjtgs95h4t80000gp/T/ipykernel_25334/4138346147.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  happiness_countries_merged_df = happiness_countries_merged_df.append(new_row, ignore_index=True)


,Country,ISO_A3,ISO_A2,geometry,Happiness Score_2015,Happiness Score_2016,Happiness Score_2017,Happiness Score_2018,Happiness Score_2019,Happiness Score_2020,Happiness Score_2021,Happiness Score_2022
0,Norway,NOR,NO,"MULTIPOLYGON (((3.38258 -54.44931, 3.36451 -54...",7.522,7.498,7.5370,7.594,7.554,7.4880,7.392,7.365
1,Denmark,DNK,DK,"MULTIPOLYGON (((11.25603 54.95458, 11.30348 54...",7.527,7.526,7.5220,7.555,7.600,7.6456,7.620,7.636
2,Iceland,ISL,IS,"MULTIPOLYGON (((-20.26964 63.40913, -20.27953 ...",7.561,7.501,7.5040,7.495,7.494,7.5045,7.554,7.557
3,Switzerland,CHE,CH,"MULTIPOLYGON (((8.61744 47.75732, 8.62984 47.7...",7.587,7.509,7.4940,7.487,7.480,7.5599,7.571,7.512
4,Finland,FIN,FI,"MULTIPOLYGON (((22.44370 59.85147, 22.42774 59...",7.406,7.413,7.4690,7.632,7.769,7.8087,7.842,7.821
...,...,...,...,...,...,...,...,...,...,...,...,...
149,Syria,SYR,SY,"MULTIPOLYGON (((42.23683 37.28630, 42.26722 37...",3.006,3.069,3.4620,3.462,3.462,NaN,NaN,<NA>
150,Tanzania,NaN,NaN,None,3.781,3.666,3.3490,3.303,3.231,3.4762,3.623,3.702
151,Burundi,BDI,BI,"MULTIPOLYGON (((30.41507 -2.31309, 30.41848 -2...",2.905,2.905,2.9050,2.905,3.775,NaN,NaN,<NA>
152,Central African Republic,CAF,CF,"MULTIPOLYGON (((22.55576 10.97897, 22.57705 10...",NaN,NaN,2.6930,NaN,NaN,NaN,NaN,<NA>


In [8]:
# Hong Kong's original name was too long so I changed it to "Hong Kong"
happiness_countries_merged_df.at[70,'Country'] = 'Hong Kong'

In [10]:
happiness_countries_merged_df

,Country,ISO_A3,ISO_A2,geometry,Happiness Score_2015,Happiness Score_2016,Happiness Score_2017,Happiness Score_2018,Happiness Score_2019,Happiness Score_2020,Happiness Score_2021,Happiness Score_2022
0,Norway,NOR,NO,"MULTIPOLYGON (((3.38258 -54.44931, 3.36451 -54...",7.522,7.498,7.5370,7.594,7.554,7.4880,7.392,7.365
1,Denmark,DNK,DK,"MULTIPOLYGON (((11.25603 54.95458, 11.30348 54...",7.527,7.526,7.5220,7.555,7.600,7.6456,7.620,7.636
2,Iceland,ISL,IS,"MULTIPOLYGON (((-20.26964 63.40913, -20.27953 ...",7.561,7.501,7.5040,7.495,7.494,7.5045,7.554,7.557
3,Switzerland,CHE,CH,"MULTIPOLYGON (((8.61744 47.75732, 8.62984 47.7...",7.587,7.509,7.4940,7.487,7.480,7.5599,7.571,7.512
4,Finland,FIN,FI,"MULTIPOLYGON (((22.44370 59.85147, 22.42774 59...",7.406,7.413,7.4690,7.632,7.769,7.8087,7.842,7.821
...,...,...,...,...,...,...,...,...,...,...,...,...
149,Syria,SYR,SY,"MULTIPOLYGON (((42.23683 37.28630, 42.26722 37...",3.006,3.069,3.4620,3.462,3.462,NaN,NaN,<NA>
150,Tanzania,NaN,NaN,None,3.781,3.666,3.3490,3.303,3.231,3.4762,3.623,3.702
151,Burundi,BDI,BI,"MULTIPOLYGON (((30.41507 -2.31309, 30.41848 -2...",2.905,2.905,2.9050,2.905,3.775,NaN,NaN,<NA>
152,Central African Republic,CAF,CF,"MULTIPOLYGON (((22.55576 10.97897, 22.57705 10...",NaN,NaN,2.6930,NaN,NaN,NaN,NaN,<NA>


At this point there are still a few countries that have happiness scores but aren't on the map, so we're going to find their geojson files and add them to the happiness dataframe. 

In [11]:
countries_missing_geometry_df = happiness_countries_merged_df[happiness_countries_merged_df['geometry']==None]
countries_missing_geometry_df

,Country,ISO_A3,ISO_A2,geometry,Happiness Score_2015,Happiness Score_2016,Happiness Score_2017,Happiness Score_2018,Happiness Score_2019,Happiness Score_2020,Happiness Score_2021,Happiness Score_2022
32,Taiwan Province of China,NaN,NaN,None,NaN,NaN,6.4220,NaN,NaN,NaN,NaN,<NA>
60,North Cyprus,NaN,NaN,None,5.695,5.771,5.8100,NaN,NaN,NaN,NaN,<NA>
70,Hong Kong,NaN,NaN,None,NaN,NaN,5.4720,NaN,NaN,NaN,NaN,<NA>
72,Serbia,NaN,NaN,None,5.123,5.177,5.3950,5.398,5.603,5.7782,6.078,6.178
102,Palestinian Territories,NaN,NaN,None,4.715,4.754,4.7750,4.743,4.696,4.5528,4.517,4.483
150,Tanzania,NaN,NaN,None,3.781,3.666,3.3490,3.303,3.231,3.4762,3.623,3.702
153,Congo,NaN,NaN,None,4.253,4.254,4.2855,4.402,4.615,NaN,NaN,<NA>


In [12]:
# Downloaded the geojson files for the countries that countries.geojson didn't have and add them into countries_missing_geometry_df

missing_countries_path = 'Datasets/geoJSONs/OG_Missing_Countries/'
for _, row in countries_missing_geometry_df.iterrows():
    country = row['Country']
    country_path = country.replace(' ', '_')
    # Read the corresponding geoJSON flie
    
    country_geojson = os.path.join(missing_countries_path,f'{country_path}.geojson')

    if os.path.exists(country_geojson):
        country_gdf = gpd.read_file(country_geojson)
        multi_poly = country_gdf.unary_union
        new_gdf = gpd.GeoDataFrame(geometry=[multi_poly], crs=country_gdf.crs)
        country_geometry = new_gdf['geometry'].iloc[0]
        happiness_countries_merged_df.loc[happiness_countries_merged_df['Country'] == country, 'geometry'] = country_geometry
    else:
        country_geojson = os.path.join(missing_countries_path,f'{country_path}.json')
        country_gdf = gpd.read_file(country_geojson)
        multi_poly = country_gdf.unary_union
        new_gdf = gpd.GeoDataFrame(geometry=[multi_poly], crs=country_gdf.crs)
        country_geometry = new_gdf['geometry'].iloc[0]
        happiness_countries_merged_df.loc[happiness_countries_merged_df['Country'] == country, 'geometry'] = country_geometry


In [13]:
# Convert the GeoDataFrame to a long format DataFrame
happiness_long_df = happiness_countries_merged_df.melt(
    id_vars=['Country', 'ISO_A3', 'ISO_A2', 'geometry'],
    value_vars=['Happiness Score 2015', 'Happiness Score 2016', 'Happiness Score 2017', 'Happiness Score 2018',
                'Happiness Score 2019', 'Happiness Score 2020', 'Happiness Score 2021', 'Happiness Score 2022'],
    var_name='Year',
    value_name='Happiness Score'
)

# Extract the year from the Year column
happiness_long_df['Year'] = happiness_long_df['Year'].str.extract('(\d+)').astype(int)

# Use the data_server provider to allow Altair to work with the GeoDataFrame directly
alt.data_transformers.register('data_server', data_server)
alt.data_transformers.enable('data_server')

# Define the base chart
base_chart = alt.Chart(happiness_long_df).mark_geoshape(
    stroke='black',
    strokeWidth=0.5
).encode(
    tooltip=[
        alt.Tooltip('Country:N', title='Country'),
    ]
).project('equirectangular')

# Create a list of years for which you have happiness scores
years = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

# Create a slider for selecting the year
year_slider = alt.binding_range(min=years[0], max=years[-1], step=1)
slider_selection = alt.selection_single(bind=year_slider, fields=['Year'], name='Select', init={'Year': years[0]})

# Define the choropleth map using the year slider
map_chart = base_chart.add_selection(
    slider_selection
).transform_filter(
    slider_selection
).encode(
    color=alt.Color('Happiness Score:Q', title='Happiness Score', scale=alt.Scale(scheme='viridis')),
    tooltip=[
        alt.Tooltip('Country:N', title='Country'),
        alt.Tooltip('Happiness Score:Q', title='Happiness Score')
    ]
).properties(
    title="Happiness Score by Country",
    width = 800, 
    height = 600
)

# Display the map
map_chart


alt.Chart(...)

In [36]:
hdf = happiness_df[['Country', 'Happiness Score 2019', 'Happiness Score 2021']]
hdf[hdf.isna().any(axis=1)]

,Country,Happiness Score 2019,Happiness Score 2021
22,Czech Republic,6.852,NaN
32,Taiwan Province of China,NaN,NaN
34,Qatar,6.374,NaN
37,Trinidad and Tobago,NaN,NaN
49,Belize,NaN,NaN
60,North Cyprus,NaN,NaN
70,"Hong Kong S.A.R., China",NaN,NaN
91,Macedonia,NaN,NaN
92,Somalia,NaN,NaN
96,Bhutan,5.082,NaN


In [37]:
hdf = hdf.dropna()
hdf['Hap Diff'] = hdf['Happiness Score 2021'] - hdf['Happiness Score 2019']
hdf

,Country,Happiness Score 2019,Happiness Score 2021,Hap Diff
0,Norway,7.554,7.392,-0.162
1,Denmark,7.600,7.620,0.020
2,Iceland,7.494,7.554,0.060
3,Switzerland,7.480,7.571,0.091
4,Finland,7.769,7.842,0.073
...,...,...,...,...
147,Liberia,3.975,4.625,0.650
148,Guinea,4.534,4.984,0.450
149,Togo,4.085,4.107,0.022
150,Rwanda,3.334,3.415,0.081


# Democracy Index

In [38]:
hdf.describe()

,Happiness Score 2019,Happiness Score 2021,Hap Diff
count,133.00000,133.000000,133.000000
mean,5.52682,5.603323,0.076504
std,1.07652,1.068437,0.266768
min,3.20300,2.523000,-0.719000
25%,4.69600,4.887000,-0.045000
50%,5.52500,5.716000,0.085000
75%,6.19900,6.317000,0.237000
max,7.76900,7.842000,0.724000


In [41]:
happiness_df.describe()

,Happiness Score 2015,Happiness Score 2016,Happiness Score 2017,Happiness Score 2018,Happiness Score 2019,Happiness Score 2020,Happiness Score 2021,Happiness Score 2022
count,146.00000,150.000000,155.000000,144.000000,141.000000,133.000000,133.000000,133.000000
mean,5.39589,5.373127,5.354019,5.417924,5.485383,5.579450,5.603323,5.593203
std,1.15973,1.148077,1.131230,1.112302,1.096099,1.086815,1.068437,1.100027
min,2.83900,2.905000,2.693000,2.905000,3.203000,2.566900,2.523000,2.404000
25%,4.52600,4.397250,4.505500,4.492750,4.628000,4.814100,4.887000,4.891000
50%,5.27700,5.308500,5.279000,5.404000,5.467000,5.607500,5.716000,5.737000
75%,6.26325,6.233750,6.101500,6.194750,6.199000,6.304800,6.317000,6.341000
max,7.58700,7.526000,7.537000,7.632000,7.769000,7.808700,7.842000,7.821000


### Reflection
Brighter colors mean the country is higher on the happy index than countries shaded in darker colors. In accordance with our hypothesis, the colors of the map get darker between 2018 and 2019 due to COVID and as the world started to recover from COVID, the map got brighter. This is what we expected. Moving forward, I think we should consider using Finland, Sweden, and Norway as standards of high happiness. I don't know whether we should use Syria or Afghanistan as standards of lower happiness, though. It might also be interesting to do a directo comparison of Russia (5.468)/China (5.191) vs. Canada (7.278)/USA (6.892). I don't really know if these numbers are different enough that a comparison might yield something significant. 

# Democracy Index

In [14]:
import pandas as pd
import geopandas as gpd
import altair as alt
from altair_data_server import data_server


In [15]:
# import and fix democracy data
democracy_path = 'Datasets/Democracy_Index/democracy.csv'
democracy_df = pd.read_csv(democracy_path)
democracy_df = democracy_df[democracy_df['Year'] >= 2015]
democracy_df = democracy_df.pivot(index='Entity', columns='Year', values='democracy_eiu')
democracy_df.columns = [f'democracy_score_{col}' for col in democracy_df.columns]
democracy_df.reset_index(inplace=True)

# We don't need the scores of the continents and the world 
remove = [
    'Africa',
    'North America',
    'World',
    'South America',
    'Europe',
    'Asia',
    'Oceania'
]

democracy_df = democracy_df[~democracy_df['Entity'].isin(remove)]
democracy_countries = democracy_df['Entity'].unique()
democracy_df

,Entity,democracy_score_2015,democracy_score_2016,democracy_score_2017,democracy_score_2018,democracy_score_2019,democracy_score_2020,democracy_score_2021,democracy_score_2022
0,Afghanistan,2.77,2.55,2.55,2.97,2.85,2.85,0.32,0.32
2,Albania,5.91,5.91,5.98,5.98,5.89,6.08,6.11,6.41
3,Algeria,3.95,3.56,3.56,3.50,4.01,3.77,3.77,3.66
4,Angola,3.35,3.40,3.62,3.62,3.72,3.66,3.37,3.96
5,Argentina,7.02,6.96,6.96,7.02,7.02,6.95,6.81,6.85
...,...,...,...,...,...,...,...,...,...
168,Venezuela,5.00,4.68,3.87,3.16,2.88,2.76,2.11,2.23
169,Vietnam,3.53,3.38,3.08,3.08,3.08,2.94,2.94,2.73
171,Yemen,2.24,2.07,2.07,1.95,1.95,1.95,1.95,1.95
172,Zambia,6.28,5.99,5.68,5.61,5.09,4.86,5.72,5.80


In [16]:
# import and fix countries geojson data
countries_gpd = gpd.read_file('Datasets/geoJSONs/countries.geojson')
countries_gpd = countries_gpd.rename(columns={'ADMIN':'Entity'})
countries = countries_gpd['Entity'].unique()
countries_gpd

,Entity,ISO_A3,ISO_A2,geometry
0,Aruba,ABW,AW,"MULTIPOLYGON (((-69.99694 12.57758, -69.93639 ..."
1,Afghanistan,AFG,AF,"MULTIPOLYGON (((71.04980 38.40866, 71.05714 38..."
2,Angola,AGO,AO,"MULTIPOLYGON (((11.73752 -16.69258, 11.73851 -..."
3,Anguilla,AIA,AI,"MULTIPOLYGON (((-63.03767 18.21296, -63.09952 ..."
4,Albania,ALB,AL,"MULTIPOLYGON (((19.74777 42.57890, 19.74601 42..."
...,...,...,...,...
250,Samoa,WSM,WS,"MULTIPOLYGON (((-171.57002 -13.93816, -171.564..."
251,Yemen,YEM,YE,"MULTIPOLYGON (((53.30824 12.11839, 53.31027 12..."
252,South Africa,ZAF,ZA,"MULTIPOLYGON (((37.86378 -46.94085, 37.83644 -..."
253,Zambia,ZMB,ZM,"MULTIPOLYGON (((31.11984 -8.61663, 31.14102 -8..."


In [17]:
# all the countries that are in countries but not democracy_countries
diff1 = list(set(democracy_countries) - set(countries))
diff1

['North Macedonia',
 "Cote d'Ivoire",
 'Czechia',
 'Democratic Republic of Congo',
 'Guinea-Bissau',
 'Congo',
 'Eswatini',
 'Hong Kong',
 'Serbia',
 'Tanzania',
 'Timor']

In [18]:
# the two dataframes have the same country under different names, so reconcile that
mapping_dict = {
    "United Republic of Tanzania" : 'Tanzania',
    'Czech Republic' : 'Czechia',
    'Swaziland' : 'Eswatini',
    'Democratic Republic of the Congo' : 'Democratic Republic of Congo',
    'Republic of Congo' : 'Congo',
    'Hong Kong S.A.R.' : 'Hong Kong',
    'Guinea Bissau' : 'Guinea-Bissau',
    'Macedonia' : 'North Macedonia',
    'East Timor' : 'Timor',
    'Ivory Coast' : "Cote d'Ivoire",
    'Oceania' : "Australia",
    'Republic of Serbia' : 'Serbia'
}

corrected_list = [mapping_dict.get(country, country) for country in countries]


In [19]:
# all the countries that are in corrected list but not in democracy_countries
diff2 = list(set(democracy_countries) - set(corrected_list))
print(diff2)


[]


In [20]:
countries_gpd['Entity'] = countries_gpd['Entity'].replace(mapping_dict)

In [21]:
democracy_merged = countries_gpd.merge(democracy_df, on='Entity', how='right')
democracy_merged

,Entity,ISO_A3,ISO_A2,geometry,democracy_score_2015,democracy_score_2016,democracy_score_2017,democracy_score_2018,democracy_score_2019,democracy_score_2020,democracy_score_2021,democracy_score_2022
0,Afghanistan,AFG,AF,"MULTIPOLYGON (((71.04980 38.40866, 71.05714 38...",2.77,2.55,2.55,2.97,2.85,2.85,0.32,0.32
1,Albania,ALB,AL,"MULTIPOLYGON (((19.74777 42.57890, 19.74601 42...",5.91,5.91,5.98,5.98,5.89,6.08,6.11,6.41
2,Algeria,DZA,DZ,"MULTIPOLYGON (((8.60251 36.93951, 8.60566 36.9...",3.95,3.56,3.56,3.50,4.01,3.77,3.77,3.66
3,Angola,AGO,AO,"MULTIPOLYGON (((11.73752 -16.69258, 11.73851 -...",3.35,3.40,3.62,3.62,3.72,3.66,3.37,3.96
4,Argentina,ARG,AR,"MULTIPOLYGON (((-68.65412 -54.88624, -68.65414...",7.02,6.96,6.96,7.02,7.02,6.95,6.81,6.85
...,...,...,...,...,...,...,...,...,...,...,...,...
162,Venezuela,VEN,VE,"MULTIPOLYGON (((-61.39027 8.58058, -61.36986 8...",5.00,4.68,3.87,3.16,2.88,2.76,2.11,2.23
163,Vietnam,VNM,VN,"MULTIPOLYGON (((106.66871 8.75349, 106.66586 8...",3.53,3.38,3.08,3.08,3.08,2.94,2.94,2.73
164,Yemen,YEM,YE,"MULTIPOLYGON (((53.30824 12.11839, 53.31027 12...",2.24,2.07,2.07,1.95,1.95,1.95,1.95,1.95
165,Zambia,ZMB,ZM,"MULTIPOLYGON (((31.11984 -8.61663, 31.14102 -8...",6.28,5.99,5.68,5.61,5.09,4.86,5.72,5.80


In [22]:
# Convert the GeoDataFrame to a long format DataFrame
democracy_long_df = democracy_merged.melt(
    id_vars=['Entity', 'ISO_A3', 'ISO_A2', 'geometry'],
    value_vars=['democracy_score_2015', 'democracy_score_2016',
                'democracy_score_2017', 'democracy_score_2018','democracy_score_2019', 'democracy_score_2020', 'democracy_score_2021','democracy_score_2022'],
    var_name='Year',
    value_name='Democracy Score'
)

# Extract the year from the Year column
democracy_long_df['Year'] = democracy_long_df['Year'].str.extract('(\d+)').astype(int)

# Use the data_server provider to allow Altair to work with the GeoDataFrame directly
alt.data_transformers.register('data_server', data_server)
alt.data_transformers.enable('data_server')

# Define the base chart
base_chart = alt.Chart(democracy_long_df).mark_geoshape(
    stroke='black',
    strokeWidth=0.5
).encode(
    tooltip=[
        alt.Tooltip('Entity:N', title='Country'),
    ]
).project('equirectangular')

# Create a list of years for which you have happiness scores
years = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

# Create a slider for selecting the year
year_slider = alt.binding_range(min=years[0], max=years[-1], step=1)
slider_selection = alt.selection_single(bind=year_slider, fields=['Year'], name='Select', init={'Year': years[0]})

# Define the choropleth map using the year slider
map_chart = base_chart.add_selection(
    slider_selection
).transform_filter(
    slider_selection
).encode(
    color=alt.Color('Democracy Score:Q', title='Democracy Score', scale=alt.Scale(scheme='viridis')),
    tooltip=[
        alt.Tooltip('Entity:N', title='Country'),
        alt.Tooltip('Democracy Score:Q', title='Democracy Score')
    ]
).properties(
    title="Democracy Score by Country",
    width = 800, 
    height = 600
)

# Display the map
map_chart


alt.Chart(...)

# Control Variables
- [GDP per capita](https://data.worldbank.org/indicator/NY.GDP.PCAP.PP.CD)
- [Healthy life expectancy](https://apps.who.int/gho/data/view.main.SDG2016LEXv?lang=en)
- Social support (from world happiness report data)
- Freedom (from world happiness report data)
- Generosity
- Corruption

## GDP Per Capita in terms  of Purchasing Power Parity adjusted to constant dollars

In [1]:
import pandas as pd
import altair as alt

In [44]:
gdp_path = 'Datasets/Control_Variables/GDP_per_capita_PPP_(current_international_$).csv'
gdp_df = pd.read_csv(gdp_path)

gdp_df.drop(columns = ['Country Code','Indicator Name','Indicator Code'], inplace = True)

gdp_df.dropna(how='all', axis=1, inplace=True)

columns_to_drop = list(range(1990, 2019))
columns_to_drop.append(2020)
columns_to_drop = [str(year) for year in columns_to_drop]

gdp_df.drop(columns=columns_to_drop, inplace=True)

gdp_df.dropna(subset=['2019', '2021'], inplace=True)

gdp_df.reset_index(drop=True, inplace=True)

columns_to_round = ['2019', '2021']
gdp_df[columns_to_round] = gdp_df[columns_to_round].round(2)

gdp_df

,Country Name,2019,2021
0,Aruba,42501.64,42698.36
1,Africa Eastern and Southern,3777.97,3839.47
2,Afghanistan,2167.70,1665.81
3,Africa Western and Central,4264.73,4409.45
4,Angola,6881.08,6491.13
...,...,...,...
233,Samoa,6613.98,6079.76
234,Kosovo,11796.15,13055.90
235,South Africa,14436.83,14624.42
236,Zambia,3514.69,3555.92


In [45]:
gdp_df['gdp_dif'] = gdp_df['2021'] - gdp_df['2019']
gdp_df.describe()

,2019,2021,gdp_dif
count,238.000000,238.000000,238.000000
mean,21746.273782,22348.923319,602.649538
std,22320.740815,22883.622379,4416.508074
min,760.450000,774.540000,-56820.120000
25%,5303.707500,5530.602500,11.225000
50%,14535.645000,15041.575000,350.860000
75%,31475.577500,31824.745000,1343.100000
max,128005.780000,133329.820000,15684.330000


## Healthy Life Expectancy

The WHO dataset only gives us data 2005, 2010, 2015, and 2019. So, to get 2019, 2020, and 2021, I'll interpolate the data. This is the same approach used in the World Happiness Report as they reported [here](https://happiness-report.s3.amazonaws.com/2022/Appendix_1_StatiscalAppendix_Ch2.pdf)

In [58]:
import numpy as np
from scipy.interpolate import BarycentricInterpolator
import pandas as pd 

In [59]:
hle_path = 'Datasets/Control_Variables/HLE_by_country.csv'
hle_df = pd.read_csv(hle_path)

hle_df.rename(columns={'Unnamed: 0':'Country', 'Unnamed: 1':'Year'}, inplace=True)
selected_columns = [
    'Country',
    'Year',
    'Life expectancy at birth (years)',
    'Life expectancy at age 60 (years)',
    'Healthy life expectancy (HALE) at birth (years)',
    'Healthy life expectancy (HALE) at age 60 (years)',
]
# this is for both sexes
hle_df = hle_df[selected_columns]
hle_df.drop(0, inplace=True)

columns_to_convert = [
    'Life expectancy at birth (years)',
    'Life expectancy at age 60 (years)',
    'Healthy life expectancy (HALE) at birth (years)',
    'Healthy life expectancy (HALE) at age 60 (years)',
]
hle_df[columns_to_convert] = hle_df[columns_to_convert].astype(float)

hle_df['Year'] = hle_df['Year'].astype(int)

hle_df

,Country,Year,Life expectancy at birth (years),Life expectancy at age 60 (years),Healthy life expectancy (HALE) at birth (years),Healthy life expectancy (HALE) at age 60 (years)
1,Afghanistan,2019,63.2,15.2,53.9,10.8
2,Afghanistan,2015,61.7,15.6,52.6,11.2
3,Afghanistan,2010,59.9,15.1,51.1,10.9
4,Afghanistan,2000,55.0,13.9,46.8,10.2
5,Albania,2019,78.0,21.0,69.1,16.6
...,...,...,...,...,...,...
728,Zambia,2000,44.5,13.2,39.0,10.0
729,Zimbabwe,2019,60.7,15.6,53.1,11.5
730,Zimbabwe,2015,58.5,15.1,51.2,11.2
731,Zimbabwe,2010,51.5,14.1,45.2,10.5


In [60]:
def interpolate_hle_data(years, hle_values, target_years):
    interpolator = BarycentricInterpolator(years, hle_values)
    return interpolator(target_years)


In [61]:
countries = hle_df['Country'].unique()
target_years = [2019, 2020, 2021, 2022]


In [62]:
# Initialize an empty dictionary to store the results
estimated_hle_data = {}

for country in countries:
    # Filter data for the current country
    country_data = hle_df[hle_df['Country'] == country]
    
    # Extract years and HLE values for the current country
    years = country_data['Year'].values
    hle_birth_values = country_data['Healthy life expectancy (HALE) at birth (years)'].values.astype(float)
    hle_age_60_values = country_data['Healthy life expectancy (HALE) at age 60 (years)'].values.astype(float)
    
    # Perform interpolation for HLE at birth and age 60
    estimated_hle_birth = interpolate_hle_data(years, hle_birth_values, target_years)
    estimated_hle_age_60 = interpolate_hle_data(years, hle_age_60_values, target_years)
    
    # Store the original and interpolated results for the current country
    estimated_hle_data[country] = {
        'HALE_at_birth': dict(zip(target_years, estimated_hle_birth)),
        'HALE_at_age_60': dict(zip(target_years, estimated_hle_age_60)),
    }

print(estimated_hle_data)


{'Afghanistan': {'HALE_at_birth': {2019: 53.9, 2020: 54.269005847953224, 2021: 54.66284210526314, 2022: 55.085122807017534}, 'HALE_at_age_60': {2019: 10.8, 2020: 10.566081871345029, 2021: 10.26778947368421, 2022: 9.899719298245616}}, 'Albania': {'HALE_at_birth': {2019: 69.1, 2020: 68.9017543859649, 2021: 68.59463157894734, 2022: 68.16884210526314}, 'HALE_at_age_60': {2019: 16.6, 2020: 16.583625730994154, 2021: 16.5761052631579, 2022: 16.580140350877194}}, 'Algeria': {'HALE_at_birth': {2019: 66.4, 2020: 66.53157894736844, 2021: 66.68336842105265, 2022: 66.85915789473685}, 'HALE_at_age_60': {2019: 16.0, 2020: 16.00233918128655, 2021: 15.99484210526316, 2022: 15.977122807017546}}, 'Angola': {'HALE_at_birth': {2019: 54.8, 2020: 54.80877192982456, 2021: 54.69315789473683, 2022: 54.44421052631578}, 'HALE_at_age_60': {2019: 12.6, 2020: 12.611111111111112, 2021: 12.604, 2022: 12.577333333333325}}, 'Antigua and Barbuda': {'HALE_at_birth': {2019: 67.0, 2020: 67.09298245614036, 2021: 67.209473684

In [63]:
rows = []

for country, hle in estimated_hle_data.items():
    for year, hle_birth in hle['HALE_at_birth'].items():
        hle_age_60 = hle['HALE_at_age_60'][year]
        row = {
            'Country': country,
            'Year': year,
            'HALE_at_birth': hle_birth,
            'HALE_at_age_60': hle_age_60,
        }
        rows.append(row)


In [64]:
hle = pd.DataFrame(rows)
hle = hle.sort_values(by=['Country', 'Year']).reset_index(drop=True)
columns_to_round = ['HALE_at_birth', 'HALE_at_age_60']
hle[columns_to_round] = hle[columns_to_round].round(2)
hle

,Country,Year,HALE_at_birth,HALE_at_age_60
0,Afghanistan,2019,53.90,10.80
1,Afghanistan,2020,54.27,10.57
2,Afghanistan,2021,54.66,10.27
3,Afghanistan,2022,55.09,9.90
4,Albania,2019,69.10,16.60
...,...,...,...,...
727,Zambia,2022,55.12,12.70
728,Zimbabwe,2019,53.10,11.50
729,Zimbabwe,2020,52.82,11.49
730,Zimbabwe,2021,52.16,11.44


In [65]:
hle = hle.pivot_table(
    index='Country',
    columns='Year',
    values=['HALE_at_birth', 'HALE_at_age_60']
)
hle.reset_index(inplace=True)
hle.columns = [f"{col[0]}_{col[1]}" if col[1] else col[0] for col in hle.columns]

hle


,Country,HALE_at_age_60_2019,HALE_at_age_60_2020,HALE_at_age_60_2021,HALE_at_age_60_2022,HALE_at_birth_2019,HALE_at_birth_2020,HALE_at_birth_2021,HALE_at_birth_2022
0,Afghanistan,10.8,10.57,10.27,9.90,53.9,54.27,54.66,55.09
1,Albania,16.6,16.58,16.58,16.58,69.1,68.90,68.59,68.17
2,Algeria,16.0,16.00,15.99,15.98,66.4,66.53,66.68,66.86
3,Angola,12.6,12.61,12.60,12.58,54.8,54.81,54.69,54.44
4,Antigua and Barbuda,15.8,15.80,15.80,15.81,67.0,67.09,67.21,67.35
...,...,...,...,...,...,...,...,...,...
178,Venezuela (Bolivarian Republic of),16.3,16.28,16.27,16.27,64.4,64.13,63.81,63.45
179,Viet Nam,14.8,14.81,14.81,14.80,65.3,65.42,65.56,65.71
180,Yemen,13.3,13.18,13.02,12.83,57.5,57.20,56.87,56.52
181,Zambia,12.6,12.65,12.68,12.70,54.4,54.69,54.93,55.12


In [66]:
# For hle_60 DataFrame
hle_60_columns = ['Country', 'HALE_at_age_60_2019', 'HALE_at_age_60_2020', 'HALE_at_age_60_2021', 'HALE_at_age_60_2022']
hle_60 = hle[hle_60_columns]
hle_60 = hle_60.rename(columns={
    'HALE_at_age_60_2019': '2019',
    'HALE_at_age_60_2020': '2020',
    'HALE_at_age_60_2021': '2021',
    'HALE_at_age_60_2022': '2022'
})


# For hle_birth DataFrame
hle_birth_columns = ['Country', 'HALE_at_birth_2019', 'HALE_at_birth_2020', 'HALE_at_birth_2021', 'HALE_at_birth_2022']
hle_birth = hle[hle_birth_columns]
hle_birth = hle_birth.rename(columns={
    'HALE_at_birth_2019': '2019',
    'HALE_at_birth_2020': '2020',
    'HALE_at_birth_2021': '2021',
    'HALE_at_birth_2022': '2022'
})


In [67]:
hle_60

,Country,2019,2020,2021,2022
0,Afghanistan,10.8,10.57,10.27,9.90
1,Albania,16.6,16.58,16.58,16.58
2,Algeria,16.0,16.00,15.99,15.98
3,Angola,12.6,12.61,12.60,12.58
4,Antigua and Barbuda,15.8,15.80,15.80,15.81
...,...,...,...,...,...
178,Venezuela (Bolivarian Republic of),16.3,16.28,16.27,16.27
179,Viet Nam,14.8,14.81,14.81,14.80
180,Yemen,13.3,13.18,13.02,12.83
181,Zambia,12.6,12.65,12.68,12.70


In [68]:
hle_birth

,Country,2019,2020,2021,2022
0,Afghanistan,53.9,54.27,54.66,55.09
1,Albania,69.1,68.90,68.59,68.17
2,Algeria,66.4,66.53,66.68,66.86
3,Angola,54.8,54.81,54.69,54.44
4,Antigua and Barbuda,67.0,67.09,67.21,67.35
...,...,...,...,...,...
178,Venezuela (Bolivarian Republic of),64.4,64.13,63.81,63.45
179,Viet Nam,65.3,65.42,65.56,65.71
180,Yemen,57.5,57.20,56.87,56.52
181,Zambia,54.4,54.69,54.93,55.12


In [88]:
hle_birth1 = hle_birth[['Country', '2019', '2021']]
hle_birth1['dif'] = hle_birth1['2021'] - hle_birth1['2019']
hle_birth1[['Country', 'dif']].head(50)

<ipython-input-88-dd129c6e34d5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hle_birth1['dif'] = hle_birth1['2021'] - hle_birth1['2019']


,Country,dif
0,Afghanistan,0.76
1,Albania,-0.51
2,Algeria,0.28
3,Angola,-0.11
4,Antigua and Barbuda,0.21
5,Argentina,-0.05
6,Armenia,0.57
7,Australia,0.18
8,Austria,0.34
9,Azerbaijan,-0.06


In [85]:
hle_birth1.min()

Country    Afghanistan
2019             44.20
2021             42.35
dif             -13.45
dtype: object

,Country,2019,2021,dif


In [75]:
hle_birth1.describe()

,2019,2021,dif
count,183.000000,183.000000,183.000000
mean,63.318033,63.659235,0.341202
std,6.207037,6.228616,1.441700
min,44.200000,42.350000,-13.450000
25%,58.000000,58.375000,0.025000
50%,64.500000,65.040000,0.300000
75%,67.700000,68.015000,0.605000
max,74.100000,74.370000,11.400000


In [13]:
hle_60.to_csv('Output_Datasets/Controls/hle_60.csv', index=False)
hle_birth.to_csv('Output_Datasets/Controls/hle_birth.csv', index=False)

## Social Support

In [8]:
import pandas as pd
import os

In [11]:
# Create Social Support Data Frame
pattern = r"Social Support"
social_support_df = filtered_dataframes[0][['Country']]
for i, df in enumerate(filtered_dataframes):
    # Extract the country and happiness score columns
    filtered_columns = df.filter(regex=pattern)

    country_ss_df = pd.concat([df['Country'], filtered_columns], axis=1)

    # Merge the result DataFrame with the country_happiness_df on the 'Country' column
    social_support_df = social_support_df.merge(country_ss_df, on='Country', how='right')

# Split the DataFrame into the first column and the remaining columns
first_column = social_support_df.iloc[:, 0:1]
remaining_columns = social_support_df.iloc[:, 1:]

# Sort the remaining columns
sorted_remaining_columns = remaining_columns.sort_index(axis=1)

# Concatenate the first column with the sorted remaining columns
social_support_df = pd.concat([first_column, sorted_remaining_columns], axis=1)

social_support_df.dropna(subset=['Social Support 2018','Social Support 2019','Social Support 2020','Social Support 2021'], inplace=True)
social_support_df.reset_index(drop=True, inplace=True)

social_support_df

,Country,Social Support 2018,Social Support 2019,Social Support 2020,Social Support 2021
0,Norway,1.582,1.582,0.952487,0.954
1,Denmark,1.590,1.573,0.955991,0.954
2,Iceland,1.644,1.624,0.974670,0.983
3,Switzerland,1.549,1.526,0.942847,0.942
4,Finland,1.592,1.587,0.954330,0.954
...,...,...,...,...,...
128,Liberia,0.858,0.922,0.709281,0.720
129,Guinea,0.792,0.829,0.637573,0.639
130,Togo,0.474,0.572,0.551313,0.569
131,Rwanda,0.896,0.711,0.540835,0.552


In [12]:
social_support_df.to_csv('Output_Datasets/Controls/social_support.csv', index=False)

## Freedom

In [4]:
import pandas as pd
import os

In [5]:
# Create Social Support Data Frame
pattern = r"Freedom"
freedom_df = filtered_dataframes[0][['Country']]
for i, df in enumerate(filtered_dataframes):
    # Extract the country and happiness score columns
    filtered_columns = df.filter(regex=pattern)

    country_ss_df = pd.concat([df['Country'], filtered_columns], axis=1)

    # Merge the result DataFrame with the country_happiness_df on the 'Country' column
    freedom_df = freedom_df.merge(country_ss_df, on='Country', how='right')

# Split the DataFrame into the first column and the remaining columns
first_column = freedom_df.iloc[:, 0:1]
remaining_columns = freedom_df.iloc[:, 1:]

# Sort the remaining columns
sorted_remaining_columns = remaining_columns.sort_index(axis=1)

# Concatenate the first column with the sorted remaining columns
freedom_df = pd.concat([first_column, sorted_remaining_columns], axis=1)

relevant_years = ['Country','Freedom 2018','Freedom 2019','Freedom 2020','Freedom 2021']

freedom_df.dropna(subset=relevant_years, inplace=True)
freedom_df.reset_index(drop=True, inplace=True)


freedom_df = freedom_df[relevant_years]
freedom_df

,Country,Freedom 2018,Freedom 2019,Freedom 2020,Freedom 2021
0,Norway,0.686,0.603,0.955750,0.960
1,Denmark,0.683,0.592,0.951444,0.946
2,Iceland,0.677,0.591,0.948892,0.955
3,Switzerland,0.660,0.572,0.921337,0.919
4,Finland,0.681,0.596,0.949172,0.949
...,...,...,...,...,...
128,Liberia,0.419,0.370,0.735269,0.735
129,Guinea,0.394,0.332,0.706847,0.697
130,Togo,0.434,0.293,0.649829,0.619
131,Rwanda,0.636,0.555,0.900589,0.897


In [6]:
freedom_df.to_csv('Output_Datasets/Controls/freedom.csv', index=False)

## Generosity

In [4]:
# Create Social Support Data Frame
pattern = r"Generosity"
generosity_df = filtered_dataframes[0][['Country']]
for i, df in enumerate(filtered_dataframes):
    # Extract the country and happiness score columns
    filtered_columns = df.filter(regex=pattern)

    country_ss_df = pd.concat([df['Country'], filtered_columns], axis=1)

    # Merge the result DataFrame with the country_happiness_df on the 'Country' column
    generosity_df = generosity_df.merge(country_ss_df, on='Country', how='right')

# Split the DataFrame into the first column and the remaining columns
first_column = generosity_df.iloc[:, 0:1]
remaining_columns = generosity_df.iloc[:, 1:]

# Sort the remaining columns
sorted_remaining_columns = remaining_columns.sort_index(axis=1)

# Concatenate the first column with the sorted remaining columns
generosity_df = pd.concat([first_column, sorted_remaining_columns], axis=1)

relevant_years = ['Country','Generosity 2018','Generosity 2019','Generosity 2020','Generosity 2021']

generosity_df.dropna(subset=relevant_years[1:], inplace=True)
generosity_df.reset_index(drop=True, inplace=True)


generosity_df = generosity_df[relevant_years]
generosity_df

,Country,Generosity 2018,Generosity 2019,Generosity 2020,Generosity 2021
0,Norway,0.286,0.271,0.134533,0.093
1,Denmark,0.284,0.252,0.066202,0.030
2,Iceland,0.353,0.354,0.246944,0.160
3,Switzerland,0.256,0.263,0.105911,0.025
4,Finland,0.202,0.153,-0.059482,-0.098
...,...,...,...,...,...
128,Liberia,0.206,0.233,0.042273,0.050
129,Guinea,0.185,0.207,0.076328,0.095
130,Togo,0.158,0.177,0.002668,0.032
131,Rwanda,0.200,0.217,0.055484,0.061


In [5]:
generosity_df.to_csv('Output_Datasets/Controls/generosity.csv', index=False)

## Corruption

In [6]:
# Create Social Support Data Frame
pattern = r"Corruption"
corruption_df = filtered_dataframes[0][['Country']]
for i, df in enumerate(filtered_dataframes):
    # Extract the country and happiness score columns
    filtered_columns = df.filter(regex=pattern)

    country_ss_df = pd.concat([df['Country'], filtered_columns], axis=1)

    # Merge the result DataFrame with the country_happiness_df on the 'Country' column
    corruption_df = corruption_df.merge(country_ss_df, on='Country', how='right')

# Split the DataFrame into the first column and the remaining columns
first_column = corruption_df.iloc[:, 0:1]
remaining_columns = corruption_df.iloc[:, 1:]

# Sort the remaining columns
sorted_remaining_columns = remaining_columns.sort_index(axis=1)

# Concatenate the first column with the sorted remaining columns
corruption_df = pd.concat([first_column, sorted_remaining_columns], axis=1)

relevant_years = ['Country','Corruption 2018','Corruption 2019','Corruption 2020','Corruption 2021']

corruption_df.dropna(subset=relevant_years[1:], inplace=True)
corruption_df.reset_index(drop=True, inplace=True)


corruption_df = corruption_df[relevant_years]
corruption_df

,Country,Corruption 2018,Corruption 2019,Corruption 2020,Corruption 2021
0,Norway,0.340,0.341,0.263218,0.270
1,Denmark,0.408,0.410,0.168489,0.179
2,Iceland,0.138,0.118,0.711710,0.673
3,Switzerland,0.357,0.343,0.303728,0.292
4,Finland,0.393,0.393,0.195445,0.186
...,...,...,...,...,...
127,Liberia,0.030,0.033,0.856376,0.850
128,Guinea,0.094,0.086,0.761794,0.766
129,Togo,0.101,0.085,0.757733,0.772
130,Rwanda,0.444,0.411,0.183541,0.167


In [7]:
corruption_df.to_csv('Output_Datasets/Controls/corruption.csv', index=False)